In [3]:
#imports
from __future__ import division, print_function
import sympy as sm
from sympy import trigsimp
import sympy.physics.mechanics as me
import mpmath as mp

from sympy.physics.vector import init_vprinting, vlatex
init_vprinting(use_latex='mathjax', pretty_print=False)

# Defining symbolic Variables

n = 1
q = me.dynamicsymbols('q:{}'.format(n + 1))  # generalized coordinates
qdot = me.dynamicsymbols('qdot:{}'.format(n + 1))  #generalized speeds
f = me.dynamicsymbols('f')
m = sm.symbols('m:{}'.format(n + 1))
J = sm.symbols('J:{}'.format(n + 1))
l = sm.symbols('l:{}'.format(n))  # lenght of each pendlum
a = sm.symbols('a:{}'.format(n))  #location of Mass-centers
d = sm.symbols('d1:{}'.format(n + 1))  #viscous damping coef.
g, t = sm.symbols('g t')

# inertial reference frame
In_frame = me.ReferenceFrame('In_frame')

# Origninal Point O in Reference Frame :
O = me.Point('O')
O.set_vel(In_frame, 0)

# The Center of Mass Point on cart :
C0 = me.Point('C0')
C0.set_pos(O, q[0] * In_frame.x)
C0.set_vel(In_frame, qdot[0] * In_frame.x)

cart_inertia_dyadic = me.inertia(In_frame, 0, 0, J[0])
cart_central_inertia = (cart_inertia_dyadic, C0)

cart = me.RigidBody('Cart', C0, In_frame, m[0], cart_central_inertia)

kindiffs = [q[0].diff(t) - qdot[0]]  # entforcing qdot=Omega

frames = [In_frame]
mass_centers = [C0]
joint_centers = [C0]
central_inertias = [cart_central_inertia]
forces = [(C0, f * In_frame.x - m[0] * g * In_frame.y)]

#adding torques :
if n == 1:
    torqueVector = (-d[0] * qdot[1]) *(-1)* In_frame.z
elif n >= 2:
    torqueVector = (-d[0] * qdot[1] - d[1] * (qdot[1] - qdot[2])) *(-1)* In_frame.z

torques = [(In_frame, torqueVector)]

# cart_potential = 1 / 2 * d[0] * qdot[1]**2
# potentials = [cart_potential]
# cart.potential_energy= cart_potential


rigid_bodies = [cart]
# Lagrangian0 = me.Lagrangian(In_frame, rigid_bodies[0])
# Lagrangians=[Lagrangian0]


for i in range(n):
    #Creating new reference frame
    Li = In_frame.orientnew('L' + str(i), 'Axis', [sm.pi/2 - q[i + 1], In_frame.z])
    Li.set_ang_vel(In_frame, -qdot[i + 1] * In_frame.z)
    frames.append(Li)

    # Creating new mass point centers
    Pi = mass_centers[-1].locatenew('a' + str(i + 1), a[i] * Li.x)
    Pi.v2pt_theory(joint_centers[-1], In_frame, Li)
    mass_centers.append(Pi)

    #Creating new joint Points
    Jointi = joint_centers[-1].locatenew('jont' + str(i + 1), l[i] * Li.x)
    Jointi.v2pt_theory(joint_centers[-1], In_frame, Li)
    joint_centers.append(Jointi)

    #adding forces
    forces.append((Pi, -m[i + 1] * g * In_frame.y))
    
    
    #adding torques
    if i==0 and n==1 :
        pass
    elif i==0 and n!=1 :
        torqueVectori= -d[0] * qdot[1] *(-1)* In_frame.z
        torques.append((Li, torqueVectori))
    else:
        torqueVectori = -d[i] * (qdot[i+1]-qdot[i]) *(-1)* In_frame.z
        torques.append((Li, torqueVectori))
    
    #adding cential inertias 
    IDi = me.inertia(frames[i + 1], 0, 0, J[i + 1])
    ICi = (IDi, mass_centers[i + 1])
    central_inertias.append(ICi)

    LBodyi = me.RigidBody('L' + str(i + 1) + '_Body', mass_centers[i + 1],
                          frames[i + 1], m[i + 1], central_inertias[i + 1])
    rigid_bodies.append(LBodyi)

    kindiffs.append(q[i + 1].diff(t) - qdot[i + 1])
    
  
   
#generalized force
torques.append((frames[1], -d[0]*qdot[1]*frames[1].z))
loads = torques + forces
#loads=forces
#Kane's Method --> Equation of motion
Kane = me.KanesMethod(In_frame, q, qdot, kd_eqs=kindiffs)
fr, frstar = Kane.kanes_equations(rigid_bodies, loads)

mass_matrix = trigsimp(Kane.mass_matrix_full)
forcing_vector = trigsimp(Kane.forcing_full)



In [4]:
loads

[(In_frame, d1*qdot1*In_frame.z),
 (L0, - d1*qdot1*L0.z),
 (C0, f*In_frame.x - g*m0*In_frame.y),
 (a1, - g*m1*In_frame.y)]

In [5]:
fr

Matrix([
[                         f],
[a0*g*m1*sin(q1) + d1*qdot1]])

In [6]:
qq=sm.Matrix([q, qdot])
equilib_point = sm.Matrix([1. ,sm.pi/6 , 1, 1])

parameter_values = [(g, 9.81), (a[0], 0.2), (d[0], 10.0), (m[0], 3.34),
                    (m[1], 0.8512), (J[0], 0), (J[1], 0.01980)]

parameters = list(map(lambda a,b :(a,b),qq, equilib_point)) + parameter_values
param_dict=dict(parameters)


In [7]:
frstar

Matrix([
[a0*m1*qdot1**2*sin(q1) - a0*m1*cos(q1)*qdot1' - (m0 + m1)*qdot0'],
[                  -a0*m1*cos(q1)*qdot0' - (J1 + a0**2*m1)*qdot1']])

In [8]:
xdot_expr=(mass_matrix.inv()*forcing_vector).expand()

In [9]:
xdot_expr.simplify()

Matrix([
[                                                                                                                                                                                     qdot0],
[                                                                                                                                                                                     qdot1],
[(J1*a0*m1*qdot1**2*sin(q1) + J1*f + a0**3*m1**2*qdot1**2*sin(q1) - a0**2*g*m1**2*sin(2*q1)/2 + a0**2*m1*f - a0*d1*m1*qdot1*cos(q1))/(J1*m0 + J1*m1 + a0**2*m0*m1 + a0**2*m1**2*sin(q1)**2)],
[        (-a0**2*m1**2*qdot1**2*sin(2*q1)/2 + a0*g*m0*m1*sin(q1) + a0*g*m1**2*sin(q1) - a0*m1*f*cos(q1) + d1*m0*qdot1 + d1*m1*qdot1)/(J1*m0 + J1*m1 + a0**2*m0*m1 + a0**2*m1**2*sin(q1)**2)]])

In [10]:
collected_qdd0=xdot_expr[2].collect(f, evaluate=False)
collected_qdd1=xdot_expr[3].collect(f, evaluate=False)

In [11]:
fx=sm.Matrix([[qdot[0]],
              [qdot[1]],
              [collected_qdd0[1]],
              [collected_qdd1[1]]])

gx=sm.Matrix([[0.0],
              [0.0],
              [collected_qdd0[f]],
              [collected_qdd1[f]]])

xx=sm.Matrix([q[0], q[1], qdot[0], qdot[1]])

In [19]:
fx[2].simplify()

a0*m1*(2*J1*qdot1**2*sin(q1) + 2*a0**2*m1*qdot1**2*sin(q1) - a0*g*m1*sin(2*q1) - 2*d1*qdot1*cos(q1))/(2*(J1*m0 + J1*m1 + a0**2*m0*m1 + a0**2*m1**2*sin(q1)**2))

In [18]:
fx[3].simplify()

(-a0**2*m1**2*qdot1**2*sin(2*q1)/2 + a0*g*m0*m1*sin(q1) + a0*g*m1**2*sin(q1) + d1*m0*qdot1 + d1*m1*qdot1)/(J1*m0 + J1*m1 + a0**2*m0*m1 + a0**2*m1**2*sin(q1)**2)

In [12]:
fx.subs(param_dict).evalf()

Matrix([
[              1.0],
[              1.0],
[-7.80993528898422],
[ 222.598107007434]])

In [13]:
gx.subs(param_dict).evalf()

Matrix([
[                 0],
[                 0],
[ 0.264023561869032],
[-0.722878570583608]])

In [14]:
xx

Matrix([
[   q0],
[   q1],
[qdot0],
[qdot1]])

In [63]:
gx_lin=gx.jacobian(xx)
fx_lin=fx.jacobian(xx)

In [64]:
fx_lin*xx

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         qdot0],
[                                            

In [65]:
gx_lin*xx

Matrix([
[                                                                                                                                                                                                                     0],
[                                                                                                                                                                                                                     0],
[(-2*J1*a0**2*m1**2*sin(q1)*cos(q1)/(J1*m0 + J1*m1 + a0**2*m0*m1 - a0**2*m1**2*cos(q1)**2 + a0**2*m1**2)**2 - 2*a0**4*m1**3*sin(q1)*cos(q1)/(J1*m0 + J1*m1 + a0**2*m0*m1 - a0**2*m1**2*cos(q1)**2 + a0**2*m1**2)**2)*q1],
[                    (2*a0**3*m1**3*sin(q1)*cos(q1)**2/(J1*m0 + J1*m1 + a0**2*m0*m1 - a0**2*m1**2*cos(q1)**2 + a0**2*m1**2)**2 + a0*m1*sin(q1)/(J1*m0 + J1*m1 + a0**2*m0*m1 - a0**2*m1**2*cos(q1)**2 + a0**2*m1**2))*q1]])

In [66]:
fx_lin.subs(param_dict)*xx

Matrix([
[               qdot0],
[               qdot1],
[-1.44535471936607*q1],
[ 35.5837094678517*q1]])

In [216]:
gx_lin.subs(param_dict).evalf()*xx

Matrix([
[                     0],
[                     0],
[-0.0324914914247285*q1],
[  0.506313632868062*q1]])